# This is the code for Quantum Optimization

In [77]:
import unittest
import qiskit
from qiskit.quantum_info import SparsePauliOp
from qiskit_optimization import QuadraticProgram
from qiskit_optimization.translators import from_docplex_mp
from docplex.mp.model import Model
import numpy as np
from qiskit_algorithms import QAOA
from qiskit_optimization.algorithms import MinimumEigenOptimizer
from qiskit_algorithms.optimizers import COBYLA
from qiskit_ibm_runtime import QiskitRuntimeService, SamplerV2 as SamplerV2
import operator
from qiskit.primitives import Sampler, Estimator
from qiskit_optimization.converters import QuadraticProgramToQubo
from qiskit.circuit.library import QAOAAnsatz
import qiskit_aer as Aer
from qiskit_ibm_runtime import Session, EstimatorV2 as Estimator
from scipy.optimize import minimize
from qiskit_aer import AerSimulator
from qiskit import transpile
from qiskit.primitives import StatevectorSampler
from qiskit_optimization.converters import InequalityToEquality
from qiskit import QuantumCircuit, transpile
from qiskit.visualization import circuit_drawer
from qiskit.transpiler import PassManager
from qiskit_aer.noise import NoiseModel, depolarizing_error, thermal_relaxation_error
from qiskit.transpiler.passes import Optimize1qGates, CXCancellation
from qiskit import QuantumCircuit, QuantumRegister
from qiskit.circuit import Parameter
import matplotlib
matplotlib.use('TkAgg')  # Set non-interactive backend
import matplotlib.pyplot as plt
from qiskit import transpile
from qiskit.visualization import circuit_drawer
from qiskit.transpiler import PassManager
from qiskit.transpiler.passes import Optimize1qGates, CXCancellation
from qiskit_ibm_runtime import QiskitRuntimeService, SamplerV2 as Sampler2
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
import os
from qiskit_algorithms import VQE
from qiskit.primitives import Estimator as Estimator3
from qiskit.circuit.library import TwoLocal
from qiskit_algorithms.optimizers import COBYLA

In [78]:
pip show qiskit

Name: qiskit
Version: 1.2.4
Summary: An open-source SDK for working with quantum computers at the level of extended quantum circuits, operators, and primitives.
Home-page: https://www.ibm.com/quantum/qiskit
Author: 
Author-email: Qiskit Development Team <qiskit@us.ibm.com>
License: Apache 2.0
Location: c:\users\arthu\miniconda3\envs\quantumqiskit\lib\site-packages
Requires: dill, numpy, python-dateutil, rustworkx, scipy, stevedore, symengine, sympy, typing-extensions
Required-by: qiskit-aer, qiskit-algorithms, qiskit-ibm-runtime, qiskit-machine-learning, qiskit-optimization
Note: you may need to restart the kernel to use updated packages.


## We first create the Docplex/Cplex optimization problem

In [79]:
opt_model = Model(name="MIP Model")

Nc= 3 # Nc is the number of seconds

Dist=2 # Distance to travel

tolerance = 0 # Tolerance in distance travelled

delta_v=1 # Rate of acceleration/deceleration set to 1

vmax=2 # Max speed of a TGV in France (in m/s)

alpha=0.05 # Regenerative braking efficiency

"""
We define two binary variables for two bits. 
When x=0 and y=0 then constant velocity
When x=1 and y=0 then acceleration
When x=0 and y=1 then breaking
"""

x={}
for i in range(0, Nc):
    x[i]= opt_model.binary_var(name=f"x_{i}")

y={}
for i in range(0, Nc):
    y[i]= opt_model.binary_var(name=f"y_{i}")

'''z={}
for i in range(0, Nc):
    z[i]= opt_model.binary_var(name=f"z_{i}")'''
    

objective = opt_model.linear_expr()
## objective is the hamiltonian/energy value we want to minimize
## Energy:
for i in range(0, Nc):
    '''objective += (delta_v**2) * x[i] - alpha*(delta_v**2)*y[i]'''
    objective += (delta_v**2)*x[i]

    
opt_model.minimize(objective)

In [80]:
## Constraint 1: (simultaneous braking/acceleration)


'''for i in range(0, Nc):
    opt_model.add_constraint(z[i] <= x[i] , f"z_u_d_{i}") 

for i in range(0, Nc):
    opt_model.add_constraint(z[i] <= y[i], f"z_p_d_{i}")
        
for i in range(0, Nc):
    opt_model.add_constraint(z[i] >= x[i]+ y[i] -1 , f"z_u_p_d_{i}") 

for i in range(0,Nc):
    opt_model.add_constraint(z[i] == 0, "No simultaneous braking or acceleration constraint"+str(i))
opt_model.add_constraint(opt_model.sum(z[i] for i in range(0, Nc)) == 0 , "No_simultaneous_braking_or_acceleration_constraint")
'''

for i in range(0, Nc):
    opt_model.add_constraint(x[i]+ y[i] -1 <= 0 , f"simultaneous_braking_acceleration_constraint_{i}") 
#### Print the optimization model info

opt_model.print_information()
opt_model.prettyprint()

Model: MIP Model
 - number of variables: 6
   - binary=6, integer=0, continuous=0
 - number of constraints: 3
   - linear=3
 - parameters: defaults
 - objective: minimize
 - problem type is: MILP
// This file has been generated by DOcplex
// model name is: MIP Model
// single vars section
dvar bool x_0;
dvar bool x_1;
dvar bool x_2;
dvar bool y_0;
dvar bool y_1;
dvar bool y_2;

minimize
 x_0 + x_1 + x_2;
 
subject to {
 simultaneous_braking_acceleration_constraint_0:
  x_0 + y_0 -1 <= 0;
 simultaneous_braking_acceleration_constraint_1:
  x_1 + y_1 -1 <= 0;
 simultaneous_braking_acceleration_constraint_2:
  x_2 + y_2 -1 <= 0;

}


In [81]:
"""
Constraint 2: (Total Distance constraints)
"""

distance = opt_model.linear_expr()
velocity = 0
for i in range(0, Nc):
    velocity = velocity + delta_v*(x[i]-y[i])
    distance += velocity
'''opt_model.add_constraint(distance <= Dist+tolerance, "Max_Distance_constraint")
opt_model.add_constraint(distance >= Dist-tolerance, "Min_Distance_constraint")'''
opt_model.add_constraint(distance == Dist, "Distance_constraint")

#### Print the optimization model info

opt_model.print_information() 

Model: MIP Model
 - number of variables: 6
   - binary=6, integer=0, continuous=0
 - number of constraints: 4
   - linear=4
 - parameters: defaults
 - objective: minimize
 - problem type is: MILP


In [82]:
"""
Constraint 3: (Net-Zero contraint)

"""
opt_model.add_constraint(opt_model.sum((y[i]-x[i]) for i in range(0, Nc)) == 0 , "Net_Zero_constraint")

#### Print the optimization model info

opt_model.print_information()

Model: MIP Model
 - number of variables: 6
   - binary=6, integer=0, continuous=0
 - number of constraints: 5
   - linear=5
 - parameters: defaults
 - objective: minimize
 - problem type is: MILP


In [83]:
"""
Constraint 4: (Maximum Speed)

"""
opt_model.add_constraint(opt_model.sum((delta_v*(x[i]) for i in range(0, Nc))) <= vmax , "Maximum_Speed_constraint")

#### Print the optimization model info

opt_model.print_information() 

Model: MIP Model
 - number of variables: 6
   - binary=6, integer=0, continuous=0
 - number of constraints: 6
   - linear=6
 - parameters: defaults
 - objective: minimize
 - problem type is: MILP


In [84]:

#Constraint 5: (Positive Speed)
for i in range(Nc):
    opt_model.add_constraint(opt_model.sum((x[i]-y[i]) for i in range(0, i)) >= 0 , "Positive_Speed_constraint"+str(i))

    #### Print the optimization model

opt_model.print_information()


Model: MIP Model
 - number of variables: 6
   - binary=6, integer=0, continuous=0
 - number of constraints: 9
   - linear=9
 - parameters: defaults
 - objective: minimize
 - problem type is: MILP


In [85]:
'''
#Constraint 6: (Must leave immediately)

opt_model.add_constraint(x[0]==1, "Must leave immediately constraint")
    

#### Print the optimization model info

opt_model.print_information()
'''

'\n#Constraint 6: (Must leave immediately)\n\nopt_model.add_constraint(x[0]==1, "Must leave immediately constraint")\n    \n\n#### Print the optimization model info\n\nopt_model.print_information()\n'

In [86]:
#### Print the optimization model
print(opt_model.prettyprint())

// This file has been generated by DOcplex
// model name is: MIP Model
// single vars section
dvar bool x_0;
dvar bool x_1;
dvar bool x_2;
dvar bool y_0;
dvar bool y_1;
dvar bool y_2;

minimize
 x_0 + x_1 + x_2;
 
subject to {
 simultaneous_braking_acceleration_constraint_0:
  x_0 + y_0 -1 <= 0;
 simultaneous_braking_acceleration_constraint_1:
  x_1 + y_1 -1 <= 0;
 simultaneous_braking_acceleration_constraint_2:
  x_2 + y_2 -1 <= 0;
 Distance_constraint:
  3 x_0 - 3 y_0 + 2 x_1 - 2 y_1 + x_2 - y_2 == 2;
 Net_Zero_constraint:
  y_0 - x_0 + y_1 - x_1 + y_2 - x_2 == 0;
 Maximum_Speed_constraint:
  x_0 + x_1 + x_2 <= 2;
 Positive_Speed_constraint0:
  0 >= 0;
 Positive_Speed_constraint1:
  x_0 - y_0 >= 0;
 Positive_Speed_constraint2:
  x_0 - y_0 + x_1 - y_1 >= 0;

}
None


## Problem conversion

In [87]:
# Conversion to qad_model
qp_quad = from_docplex_mp(opt_model)
print("Number of variables:", len(qp_quad.variables))

Number of variables: 6


C:\Users\arthu\AppData\Local\Temp\ipykernel_17824\414607200.py:2: UserWarning: Trivial constraint: Positive_Speed_constraint0: 0 >= 0
  qp_quad = from_docplex_mp(opt_model)


In [88]:
# Conversion to qubo
conv = QuadraticProgramToQubo()
qubo = conv.convert(qp_quad)

print(f"QUBO variables: {len(qubo.variables)}")
print(f"QUBO constraints: {len(qubo.linear_constraints)}")

'''opt_model.export_as_lp("qubo1.lp")'''

QUBO variables: 10
QUBO constraints: 0


'opt_model.export_as_lp("qubo1.lp")'

In [89]:
qubitOp, offset = qubo.to_ising()
print("Offset:", offset)
print("Ising Hamiltonian:")
print(str(qubitOp))
num_qubits = qubitOp.num_qubits
print(num_qubits)
print(offset)

Offset: 269.5
Ising Hamiltonian:
SparsePauliOp(['IIIIIIIIIZ', 'IIIIIIIIZI', 'IIIIIIIZII', 'IIIIIIZIII', 'IIIIIZIIII', 'IIIIZIIIII', 'IIIZIIIIII', 'IIZIIIIIII', 'IIIIIIIIZZ', 'IIIIIIIZIZ', 'IIIIIIZIIZ', 'IIIIIZIIIZ', 'IIIIZIIIIZ', 'IIIZIIIIIZ', 'IIZIIIIIIZ', 'IZIIIIIIIZ', 'IZIIIIIIII', 'ZIIIIIIIIZ', 'ZIIIIIIIII', 'IIIIIIIZZI', 'IIIIIIZIZI', 'IIIIIZIIZI', 'IIIIZIIIZI', 'IIIZIIIIZI', 'IIZIIIIIZI', 'IZIIIIIIZI', 'ZIIIIIIIZI', 'IIIIIIZZII', 'IIIIIZIZII', 'IIIIZIIZII', 'IIIZIIIZII', 'IIZIIIIZII', 'IIIIIZZIII', 'IIIIZIZIII', 'IZIIIIZIII', 'ZIIIIIZIII', 'IIIIZZIIII', 'IZIIIZIIII', 'ZIIIIZIIII', 'IIZZIIIIII', 'ZZIIIIIIII'],
              coeffs=[ 103.5+0.j,   70.5+0.j,   22.5+0.j, -114. +0.j,  -81. +0.j,  -33. +0.j,
   -8. +0.j,   -8. +0.j,   72. +0.j,   40. +0.j,  -88. +0.j,  -64. +0.j,
  -32. +0.j,    8. +0.j,    8. +0.j,   -8. +0.j,  -16. +0.j,   -8. +0.j,
  -16. +0.j,   32. +0.j,  -64. +0.j,  -47. +0.j,  -24. +0.j,    8. +0.j,
    8. +0.j,   -8. +0.j,   -8. +0.j,  -32. +0.j,  -24. +0.j,  -1

## Solving QUBO

In [90]:
p=2
qubo_circuit = QAOAAnsatz(cost_operator=qubitOp, reps=p)
qubo_circuit.measure_all()
print(qubo_circuit.num_qubits)

10


In [91]:
qubo_circuit.draw( style={'backgroundcolor': '#FFFFFF'})

┌────────────────────────────┐ ░ ┌─┐                           
    q_0: ┤0                           ├─░─┤M├───────────────────────────
         │                            │ ░ └╥┘┌─┐                        
    q_1: ┤1                           ├─░──╫─┤M├────────────────────────
         │                            │ ░  ║ └╥┘┌─┐                     
    q_2: ┤2                           ├─░──╫──╫─┤M├─────────────────────
         │                            │ ░  ║  ║ └╥┘┌─┐                  
    q_3: ┤3                           ├─░──╫──╫──╫─┤M├──────────────────
         │                            │ ░  ║  ║  ║ └╥┘┌─┐               
    q_4: ┤4                           ├─░──╫──╫──╫──╫─┤M├───────────────
         │  QAOA(γ[0],β[0],γ[1],β[1]) │ ░  ║  ║  ║  ║ └╥┘┌─┐            
    q_5: ┤5                           ├─░──╫──╫──╫──╫──╫─┤M├────────────
         │                            │ ░  ║  ║  ║  ║  ║ └╥┘┌─┐         
    q_6: ┤6                           ├─░──╫──╫──╫──╫──╫──╫─┤M├─────────
         │                            │ ░  ║  ║  ║  ║  ║  ║ └╥┘┌─┐      
    q_7: ┤7                           ├─░──╫──╫──╫──╫──╫──╫──╫─┤M├──────
         │                            │ ░  ║  ║  ║  ║  ║  ║  ║ └╥┘┌─┐   
    q_8: ┤8                           ├─░──╫──╫──╫──╫──╫──╫──╫──╫─┤M├───
         │                            │ ░  ║  ║  ║  ║  ║  ║  ║  ║ └╥┘┌─┐
    q_9: ┤9                           ├─░──╫──╫──╫──╫──╫──╫──╫──╫──╫─┤M├
         └────────────────────────────┘ ░  ║  ║  ║  ║  ║  ║  ║  ║  ║ └╥┘
meas: 10/══════════════════════════════════╩══╩══╩══╩══╩══╩══╩══╩══╩══╩═
                                           0  1  2  3  4  5  6  7  8  9

In [92]:
'''expanded_circuit = qubo_circuit.decompose()
expanded_circuit.draw( style={'backgroundcolor': '#FFFFFF'})'''

"expanded_circuit = qubo_circuit.decompose()\nexpanded_circuit.draw( style={'backgroundcolor': '#FFFFFF'})"

In [93]:
qubo_circuit.parameters

ParameterView([ParameterVectorElement(β[0]), ParameterVectorElement(β[1]), ParameterVectorElement(γ[0]), ParameterVectorElement(γ[1])])

### Optimize circuit

In [94]:
# QiskitRuntimeService.save_account(channel="ibm_quantum", token="<MY_IBM_QUANTUM_TOKEN>", overwrite=True, set_as_default=True)
'''service = QiskitRuntimeService(channel='ibm_quantum')
backend = service.least_busy(min_num_qubits=num_qubits+5)'''
backend = AerSimulator()
print(backend)

# Create pass manager for transpilation
pm = generate_preset_pass_manager(optimization_level=3,
                                    backend=backend)

candidate_circuit = pm.run(qubo_circuit)
'''candidate_circuit.draw()'''
num_qubits_circ = candidate_circuit.num_qubits
print(num_qubits_circ)

AerSimulator('aer_simulator')
10


In [95]:
import copy
from qiskit_optimization.problems.variable import VarType
from qiskit_optimization.algorithms import MinimumEigenOptimizer, CplexOptimizer

def relax_problem(problem) -> QuadraticProgram:
    """Change all variables to continuous."""
    relaxed_problem = copy.deepcopy(problem)
    for variable in relaxed_problem.variables:
        variable.vartype = VarType.CONTINUOUS

    return relaxed_problem

mu = np.array([3.418, 2.0913, 6.2415, 4.4436, 10.892, 3.4051])
sigma = np.array(
    [
        [1.07978412, 0.00768914, 0.11227606, -0.06842969, -0.01016793, -0.00839765],
        [0.00768914, 0.10922887, -0.03043424, -0.0020045, 0.00670929, 0.0147937],
        [0.11227606, -0.03043424, 0.985353, 0.02307313, -0.05249785, 0.00904119],
        [-0.06842969, -0.0020045, 0.02307313, 0.6043817, 0.03740115, -0.00945322],
        [-0.01016793, 0.00670929, -0.05249785, 0.03740115, 0.79839634, 0.07616951],
        [-0.00839765, 0.0147937, 0.00904119, -0.00945322, 0.07616951, 1.08464544],
    ]
)

qp_relax = relax_problem(QuadraticProgramToQubo().convert(qubo))
sol = CplexOptimizer().solve(qp_relax)
print(sol.prettyprint())

Error: Model has non-convex objective: 192 x_0^2 + 288 x_0 * x_1 + 160 x_0 * x_2 - 352 
objective function value: 128.0
variable values: x_0=0.0, x_1=0.0, x_2=0.0, y_0=0.0, y_1=0.0, y_2=0.0, Maximum_Speed_constraint@int_slack@0=0.0, Maximum_Speed_constraint@int_slack@1=0.0, Positive_Speed_constraint2@int_slack@0=0.0, Positive_Speed_constraint2@int_slack@1=0.0
status: FAILURE


c:\Users\arthu\miniconda3\envs\QuantumQiskit\lib\site-packages\qiskit_optimization\algorithms\cplex_optimizer.py:128: UserWarning: CPLEX cannot solve the model
  warn("CPLEX cannot solve the model")


In [96]:
c_stars = sol.samples[0].x
print(c_stars)

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


In [97]:
thetas = [2 * np.arcsin(np.sqrt(c_star)) for c_star in c_stars]

### Execute circuit

In [98]:
import random
initial_gamma = np.pi
initial_beta = np.pi/2
init_params = []
for i in range(p):
    initial_gamma = np.pi * random.random()
    initial_beta = np.pi * random.random()
    init_params.append(initial_gamma)
    init_params.append(initial_beta)

In [99]:
def cost_func_estimator(params, ansatz, hamiltonian, estimator):

    # transform the observable defined on virtual qubits to
    # an observable defined on all physical qubits
    isa_hamiltonian = hamiltonian.apply_layout(ansatz.layout)

    pub = (ansatz, isa_hamiltonian, params)
    job = estimator.run([pub])

    results = job.result()[0]
    cost = results.data.evs
    total_cost = cost + offset

    objective_func_vals.append(total_cost)


    return total_cost

In [100]:
objective_func_vals = [] # Global variable
with Session(backend=backend) as session:
    # If using qiskit-ibm-runtime<0.24.0, change `mode=` to `session=`
    estimator = Estimator(mode=session)
    estimator.options.default_shots = 10000

    # Set simple error suppression/mitigation options, this is for when on quantum hardware
    estimator.options.dynamical_decoupling.enable = True
    estimator.options.dynamical_decoupling.sequence_type = "XY4"
    estimator.options.twirling.enable_gates = True
    estimator.options.twirling.num_randomizations = "auto"

    result = minimize(
        cost_func_estimator,
        init_params,
        args=(candidate_circuit, qubitOp, estimator),
        method="COBYLA",
        tol=1e-3,
    )
    print(result)

c:\Users\arthu\miniconda3\envs\QuantumQiskit\lib\site-packages\qiskit_ibm_runtime\fake_provider\local_service.py:268: UserWarning: Options {'dynamical_decoupling': {'enable': True, 'sequence_type': 'XY4'}, 'twirling': {'enable_gates': True, 'num_randomizations': 'auto'}} have no effect in local testing mode.
  warnings.warn(f"Options {options_copy} have no effect in local testing mode.")
c:\Users\arthu\miniconda3\envs\QuantumQiskit\lib\site-packages\qiskit_ibm_runtime\fake_provider\local_service.py:268: UserWarning: Options {'dynamical_decoupling': {'enable': True, 'sequence_type': 'XY4'}, 'twirling': {'enable_gates': True, 'num_randomizations': 'auto'}} have no effect in local testing mode.
  warnings.warn(f"Options {options_copy} have no effect in local testing mode.")
c:\Users\arthu\miniconda3\envs\QuantumQiskit\lib\site-packages\qiskit_ibm_runtime\fake_provider\local_service.py:268: UserWarning: Options {'dynamical_decoupling': {'enable': True, 'sequence_type': 'XY4'}, 'twirling': 

 message: Optimization terminated successfully.
 success: True
  status: 1
     fun: 136.81349999999998
       x: [ 1.792e+00  2.229e+00  2.922e+00  4.539e+00]
    nfev: 40
   maxcv: 0.0


c:\Users\arthu\miniconda3\envs\QuantumQiskit\lib\site-packages\qiskit_ibm_runtime\fake_provider\local_service.py:268: UserWarning: Options {'dynamical_decoupling': {'enable': True, 'sequence_type': 'XY4'}, 'twirling': {'enable_gates': True, 'num_randomizations': 'auto'}} have no effect in local testing mode.
  warnings.warn(f"Options {options_copy} have no effect in local testing mode.")


In [101]:
plt.figure(figsize=(12, 6))
plt.plot(objective_func_vals)
plt.xlabel("Iteration")
plt.ylabel("Cost")
plt.show()

In [102]:
optimized_circuit = candidate_circuit.assign_parameters(result.x)

In [103]:
from qiskit_ibm_runtime import SamplerV2 as Sampler

# If using qiskit-ibm-runtime<0.24.0, change `mode=` to `backend=`
sampler = Sampler(mode=backend)
sampler.options.default_shots = 10000

# Set simple error suppression/mitigation options
sampler.options.dynamical_decoupling.enable = True
sampler.options.dynamical_decoupling.sequence_type = "XY4"
sampler.options.twirling.enable_gates = True
sampler.options.twirling.num_randomizations = "auto"

pub= (optimized_circuit, )
job = sampler.run([pub], shots=int(1e4))

counts_int = job.result()[0].data.meas.get_int_counts()
counts_bin = job.result()[0].data.meas.get_counts()
shots = sum(counts_int.values())
final_distribution_100_int = {key: val/shots for key, val in counts_int.items()}

final_distribution_int = {key: val/shots for key, val in counts_int.items()}
final_distribution_bin = {key: val/shots for key, val in counts_bin.items()}
print(final_distribution_int)
print(final_distribution_bin)

{1: 0.0098, 33: 0.0024, 13: 0.088, 157: 0.0036, 123: 0.0042, 268: 0.001, 45: 0.1263, 256: 0.0017, 75: 0.0013, 173: 0.04, 77: 0.0273, 137: 0.0101, 59: 0.0109, 105: 0.0175, 47: 0.0081, 178: 0.0017, 91: 0.0055, 121: 0.0052, 37: 0.0077, 237: 0.0083, 27: 0.0157, 238: 0.0004, 93: 0.0035, 525: 0.0066, 9: 0.0299, 109: 0.0348, 249: 0.0023, 41: 0.0193, 301: 0.0099, 255: 0.0028, 11: 0.0058, 48: 0.0016, 22: 0.0041, 283: 0.0014, 292: 0.0012, 443: 0.0009, 141: 0.0249, 5: 0.0033, 544: 0.0012, 269: 0.0073, 20: 0.0019, 573: 0.003, 15: 0.0135, 473: 0.0001, 429: 0.0028, 749: 0.0003, 397: 0.0012, 207: 0.0015, 169: 0.0179, 111: 0.0043, 0: 0.0074, 812: 0.0004, 127: 0.0068, 133: 0.0012, 175: 0.0025, 50: 0.004, 51: 0.0011, 31: 0.0165, 10: 0.0014, 904: 0.0001, 615: 0.0002, 187: 0.0047, 205: 0.0059, 296: 0.001, 559: 0.0009, 57: 0.0069, 32: 0.0042, 63: 0.0082, 34: 0.0019, 781: 0.0026, 29: 0.0112, 61: 0.0045, 307: 0.0004, 46: 0.0022, 190: 0.0008, 36: 0.0037, 512: 0.0017, 557: 0.0098, 585: 0.0007, 18: 0.008, 287: 

c:\Users\arthu\miniconda3\envs\QuantumQiskit\lib\site-packages\qiskit_ibm_runtime\fake_provider\local_service.py:268: UserWarning: Options {'dynamical_decoupling': {'enable': True, 'sequence_type': 'XY4'}, 'twirling': {'enable_gates': True, 'num_randomizations': 'auto'}} have no effect in local testing mode.
  warnings.warn(f"Options {options_copy} have no effect in local testing mode.")


In [104]:
# Sort both distributions by probability in descending order
sorted_dist_int = dict(sorted(final_distribution_int.items(), key=lambda x: x[1], reverse=True))
sorted_dist_bin = dict(sorted(final_distribution_bin.items(), key=lambda x: x[1], reverse=True))

# Print top 10 most likely outcomes
top_bin_list = []
for k, v in list(sorted_dist_bin.items())[:20]:
    temp_list = list(k)
    top_bin_list.append([int(re) for re in temp_list])


In [105]:
def calculate_cost_from_qubo(bitstring):
    # Convert bitstring to a dictionary of variable assignments
    var_assignment = {var.name: bit for var, bit in zip(qubo.variables, bitstring)}
    
    # Evaluate the cost function
    cost = qubo.objective.evaluate(var_assignment)
    
    return cost

# Example usage
cost_l = []
for bin in top_bin_list:
    cost = calculate_cost_from_qubo(bin)
    cost_l.append(cost)

min_index, min_value = min(enumerate(cost_l), key=operator.itemgetter(1))
print(min_index)

most_likely_bitstring = top_bin_list[min_index]

16


### Post-process

In [106]:
x_value = most_likely_bitstring[0:Nc]
y_value = most_likely_bitstring[Nc:2*Nc]

print(x_value)
print(y_value)

[0, 0, 1]
[0, 0, 0]


### Visualization

In [107]:
def distance(x, y, Nc):
    velocity = 0
    vel = [0]
    dist = [0]
    dist_tot= 0
    for i in range(0, Nc):
        velocity = velocity + delta_v*(x[i]-y[i])
        if velocity < 0:
            velocity = -velocity
        vel.append(velocity)
        dist_tot += velocity
        dist.append(dist_tot)
    return dist,vel

In [108]:
time= np.arange(Nc+1)
distn, velo = distance(x_value, y_value, Nc)

In [109]:
plt.figure(figsize=(15,8))

matplotlib.rcParams.update({"font.size": 15})

plt.plot(time, velo, c='b', marker="o", markersize=2, linestyle='-')#, label='label)

plt.xlabel('Time')
plt.ylabel('Velocity')
plt.title("Velocity vs Time")
plt.grid(axis='x')
plt.grid(axis='y')
plt.legend()
plt.show()

C:\Users\arthu\AppData\Local\Temp\ipykernel_17824\2353725290.py:12: UserWarning: No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
  plt.legend()


In [ ]:
plt.figure(figsize=(10,6))

matplotlib.rcParams.update({"font.size": 15})

plt.plot(time, distn, c='b', marker="o", markersize=1, linestyle='-')#, label='label')
plt.xlabel('Time')
plt.ylabel('Distance')
plt.title("Distance vs Time")
plt.legend()
plt.show()

C:\Users\arthu\AppData\Local\Temp\ipykernel_17824\1348846773.py:9: UserWarning: No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
  plt.legend()


: 